In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
df23 = pd.read_csv('2021-2023.csv',encoding='shift_jis')
df = pd.read_pickle(r"C:\Users\5D165\Desktop\20240216\df2001_2023.pkl")

In [3]:
df23.shape

(124394, 25)

In [4]:
labels=['年','日','回次','日次','レース名','確定着順','入線着順','異常コード','着差タイム','人気順','走破時計','Unnamed: 27','通過順1', '通過順2', '通過順3', '通過順4', '上がり3Fタイム','血統登録番号', '騎手コード', '調教師コード', 'レースID', '現馬主名', '生年月日', '単勝オッズ', '馬印1', 'レース印1', 'PCI', '外部指数', '外部指数差',
       '外部指数順位']

In [5]:
df.drop(labels, axis=1, inplace=True)

In [6]:
new_columns = df23.columns.to_list()

In [7]:
# カラム名を新しいリストの値に置換
df.columns = new_columns

In [8]:
lis = df.columns.to_list()
print(lis)
df.shape

['month', 'place', 'raceno', 'classcode', 'coursetype', 'coursecode', 'distancc', 'condition', 'horse', 'sex', 'age', 'jocky', 'weight', 'numberofh', 'number', 'time', 'bodyweight', 'trainer', 'affiliation', 'prize', 'producer', 'father', 'mother', 'grandfather', 'nature']


(1358108, 25)

In [9]:
df.head

<bound method NDFrame.head of         month place  raceno  classcode coursetype  coursecode  distancc  \
0           8    札幌      12         23          芝           0      1200   
1           8    札幌      12         23          芝           0      1200   
2           8    札幌      12         23          芝           0      1200   
3           8    札幌      12         23          芝           0      1200   
4           8    札幌      12         23          芝           0      1200   
...       ...   ...     ...        ...        ...         ...       ...   
241907      1    京都       1          7          ダ           1      1800   
241908      1    京都       1          7          ダ           1      1800   
241909      1    京都       1          7          ダ           1      1800   
241910      1    京都       1          7          ダ           1      1800   
241911      1    京都       1          7          ダ           1      1800   

       condition            horse sex  ...   time bodyweight  trainer

In [10]:
pd.set_option('display.max_columns', None)  # 列の省略を無効化

In [11]:
# 距離が1600の行のみ抽出
df1 = df[df['distancc'] == 1600]
df1

,month,place,raceno,classcode,coursetype,coursecode,distancc,condition,horse,sex,age,jocky,weight,numberofh,number,time,bodyweight,trainer,affiliation,prize,producer,father,mother,grandfather,nature
156,8,新潟,12,23,芝,8,1600,良,アイオブザストーム,牝,3,団野大成,53.0,18,10,92.7,468,岡田稲男,栗,800,ゴールドアップカンパニー,サトノアラジン,オーバービュー,Frankel,鹿毛
157,8,新潟,12,23,芝,8,1600,良,ニホンピロキーフ,牡,3,田口貫太,52.0,18,7,92.9,474,大橋勇樹,栗,320,友田牧場,キタサンブラック,ニホンピロアンバー,スウェプトオーヴァーボード,青鹿
158,8,新潟,12,23,芝,8,1600,良,パルティクラール,牝,3,津村明秀,53.0,18,3,93.0,502,牧浦充徳,栗,200,下河辺牧場,ヴィクトワールピサ,サンドスラッシュ,Holy Roman Emperor,黒鹿
159,8,新潟,12,23,芝,8,1600,良,ヴィアルーチェ,牝,3,石川裕紀,53.0,18,16,93.1,446,奥村武,美,120,Katsumi Yoshida,Almanzor,Via Firenze,Dansili,黒鹿
160,8,新潟,12,23,芝,8,1600,良,テイエムエメラルド,牝,5,西塚洸二,53.0,18,15,93.3,470,飯田雄三,栗,80,前川勝春,ロードカナロア,ヒミノダンス,ダンスインザダーク,黒鹿
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241768,1,京都,11,163,芝,8,1600,良,テイエムトッキュー,牡,7,和田竜二,54.0,16,11,94.4,490,岩元市三,栗,0,聖心台牧場,タマモクロス,ソシアルメーカー,マルゼンスキー,芦毛
241769,1,京都,11,163,芝,8,1600,良,シンコウスプレンダ,牡,7,横山典弘,58.0,16,3,94.4,510,古賀史生,美,0,Newgate Stud Farm Inc.,Dayjur,Over Your Shoulder,Graustark,鹿毛
241770,1,京都,11,163,芝,8,1600,良,エイシンプレストン,牡,4,福永祐一,56.0,16,16,94.5,476,北橋修二,栗,0,Joseph E. Gehl,Green Dancer,Warranty Applied,Monteverdi,鹿毛
241771,1,京都,11,163,芝,8,1600,良,オースミブライト,牡,5,村本善之,56.0,16,12,94.9,452,中尾正,栗,0,岡本牧場,ラストタイクーン,タイセイカグラ,モーニングフローリツク,黒鹿


In [12]:
# ダートのみ抽出
df3 = df1[df1['coursetype'] == 'ダ']
print(df3)

        month place  raceno  classcode coursetype  coursecode  distancc  \
5259        6    東京      10         67          ダ           1      1600   
5260        6    東京      10         67          ダ           1      1600   
5261        6    東京      10         67          ダ           1      1600   
5262        6    東京      10         67          ダ           1      1600   
5263        6    東京      10         67          ダ           1      1600   
...       ...   ...     ...        ...        ...         ...       ...   
238517      1    東京       3         15          ダ           1      1600   
238518      1    東京       3         15          ダ           1      1600   
238519      1    東京       3         15          ダ           1      1600   
238520      1    東京       3         15          ダ           1      1600   
238521      1    東京       3         15          ダ           1      1600   

       condition          horse sex  age jocky  weight  numberofh  number  \
5259           良     ワ

In [13]:
#予測ターゲットの格納
y= df3[["time"]]

#特徴量の格納
x= df3[['month','place','raceno','classcode','coursecode','condition','horse','sex','age','jocky','weight','numberofh','number','bodyweight','trainer','affiliation','prize','producer','father','mother','grandfather','nature']]

# 説明変数と目的変数
x['month'] =x['month'].astype('category')
x['place'] =x['place'].astype('category')
x['classcode'] =x['classcode'].astype('category')
x['coursecode'] =x['coursecode'].astype('category')
x['condition'] =x['condition'].astype('category')
x['horse'] =x['horse'].astype('category')
x['sex'] =x['sex'].astype('category')
x['jocky'] =x['jocky'].astype('category')
x['trainer'] =x['trainer'].astype('category')
x['affiliation'] =x['affiliation'].astype('category')
x['producer'] =x['producer'].astype('category')
x['father'] =x['father'].astype('category')
x['mother'] =x['mother'].astype('category')
x['grandfather'] =x['grandfather'].astype('category')
x['nature'] =x['nature'].astype('category')

feature_names = ['month','place','raceno','classcode','coursecode','condition','horse','sex','age','jocky','weight','numberofh','number','bodyweight','trainer','affiliation','prize','producer','father','mother','grandfather','nature']
categorical_features = ['month','place','classcode','coursecode','condition','horse','sex','jocky','trainer','affiliation','producer','father','mother','grandfather','nature']

C:\Users\5D165\AppData\Local\Temp\ipykernel_11044\4226736585.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['month'] =x['month'].astype('category')
C:\Users\5D165\AppData\Local\Temp\ipykernel_11044\4226736585.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['place'] =x['place'].astype('category')
C:\Users\5D165\AppData\Local\Temp\ipykernel_11044\4226736585.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [14]:
# 訓練データとテストデータを分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# LightGBM パラメータ
params = {
  'objective':'regression', # 目的 : 回帰
  'metric':{'rmse'}         # 評価指標 : rsme(平均二乗誤差の平方根)
}

In [ ]:
# モデルの学習
train_set = lgb.Dataset(x_train, y_train)
valid_sets = lgb.Dataset(x_test, y_test, reference=train_set)
model = lgb.train(params, train_set=train_set, valid_sets=valid_sets)

In [ ]:
# モデルをファイルに保存
model.save_model('regression_model2.txt')

# テストデータの予測
y_pred = model.predict(x_test)

In [ ]:
# 評価指標
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [ ]:
print('予測誤差 = ', rmse)
print('決定係数 = ', r2)

In [ ]:
# 散布図
plt.plot(y_test, y_test, color = 'red', label = 'x=y')
plt.scatter(y_test, y_pred)
plt.title('true vs predict')
plt.xlabel('true')
plt.ylabel('predict')

plt.show()

In [ ]:
lgb.plot_importance(model)

In [ ]:
#箱ひげ図

fig, ax = plt.subplots()
bp = ax.boxplot(df3['time'])
plt.title('time')
plt.grid() 
plt.show()

In [ ]:
df3.to_csv("0214_2.csv")

In [ ]:
#外れ値除外
#タイムが0を削除
df5 = df3[df3['time'] != 0]

In [ ]:
#箱ひげ図

fig, ax = plt.subplots()
bp = ax.boxplot(df5['time'])
plt.title('time')
plt.grid() 
plt.show()

In [ ]:
print(df5.drop(['classcode','coursecode','horse','place','prize'], axis=1))


#予測ターゲットの格納
y= df5[["time"]]

#'coursecode''horse''place''prize'削除

#特徴量の格納
x= df5[['month','raceno','condition','sex','age','jocky','weight','numberofh','number','bodyweight','trainer','affiliation','producer','father','mother','grandfather','nature']]

# 説明変数と目的変数
x['month'] =x['month'].astype('category')

x['condition'] =x['condition'].astype('category')

x['sex'] =x['sex'].astype('category')
x['jocky'] =x['jocky'].astype('category')
x['trainer'] =x['trainer'].astype('category')
x['affiliation'] =x['affiliation'].astype('category')
x['producer'] =x['producer'].astype('category')
x['father'] =x['father'].astype('category')
x['mother'] =x['mother'].astype('category')
x['grandfather'] =x['grandfather'].astype('category')
x['nature'] =x['nature'].astype('category')

feature_names = ['month','raceno','condition','sex','age','jocky','weight','numberofh','number','bodyweight','trainer','affiliation','producer','father','mother','grandfather','nature']
categorical_features = ['month','condition','sex','jocky','trainer','affiliation','producer','father','mother','grandfather','nature']


In [ ]:
# 訓練データとテストデータを分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# LightGBM パラメータ
params = {
  'objective':'regression', # 目的 : 回帰
  'metric':{'rmse'}         # 評価指標 : rsme(平均二乗誤差の平方根)
}

In [ ]:
# モデルの学習
train_set = lgb.Dataset(x_train, y_train)
valid_sets = lgb.Dataset(x_test, y_test, reference=train_set)
model = lgb.train(params, train_set=train_set, valid_sets=valid_sets)

In [ ]:
df5

In [ ]:
# モデルをファイルに保存
import pickle
with open('data.pickle', mode='wb') as f:
  pickle.dump(model, f)

In [ ]:
x_test

In [ ]:
# テストデータの予測
y_pred = model.predict(x_test)

In [ ]:
# 評価指標
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [ ]:
print('予測誤差 = ', rmse)
print('決定係数 = ', r2)

In [ ]:
# 散布図
plt.plot(y_test, y_test, color = 'red', label = 'x=y')
plt.scatter(y_test, y_pred)
plt.title('true vs predict')
plt.xlabel('true')
plt.ylabel('predict')

plt.show()

In [ ]:
lgb.plot_importance(model)

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt
import seaborn as sns

# 相関行列
correlation_coefficients = df5.corr()  # 相関行列の計算
# 相関行列のヒートマップ (相関係数の値あり) 
plt.rcParams['font.size'] = 12
plt.figure(figsize=(12, 8))  # この段階で画像のサイズを指定する
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, annot=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.show()

In [ ]:
#予測用レース結果　https://umatoku.hochi.co.jp/nar/race/2024-02-14_31_9/result

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import os
import sys


x_test = pd.read_csv('0215_prediction1.csv', encoding="shift_jis")

x_test['month'] =x_test['month'].astype('category')

x_test['condition'] =x_test['condition'].astype('category')

x_test['sex'] =x_test['sex'].astype('category')
x_test['jocky'] =x_test['jocky'].astype('category')
x_test['trainer'] =x_test['trainer'].astype('category')
x_test['affiliation'] =x_test['affiliation'].astype('category')
x_test['producer'] =x_test['producer'].astype('category')
x_test['father'] =x_test['father'].astype('category')
x_test['mother'] =x_test['mother'].astype('category')
x_test['grandfather'] =x_test['grandfather'].astype('category')
x_test['nature'] =x_test['nature'].astype('category')

feature_names = ['month','raceno','condition','sex','age','jocky','weight','numberofh','number','bodyweight','trainer','affiliation','producer','father','mother','grandfather','nature']
categorical_features = ['month','condition','sex','jocky','trainer','affiliation','producer','father','mother','grandfather','nature']




# モデル読み込み


bst = lgb.Booster(model_file='data.pickle')

# テストデータの予測
y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)

# 拡張子を除いたファイル名を取得
fname = os.path.splitext(os.path.basename(fname))[0]

# 予測値を出力
df = pd.DataFrame({'予測値':y_pred})
df.to_csv(fname + '_pred.csv', encoding='SHIFT_JIS', index=False)
